In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.FeatureCNN import FeatureCNN
from ai.DataLoader2 import DataLoader
from env import *

ETA = 1e-2
EPOCHS = 1000

print(torch.cuda.is_available())

True


In [2]:
train_dloader = DataLoader(TRAIN_DATA_PATH, TRASH_CAT, noise=True)
valid_dloader = DataLoader(VALID_DATA_PATH, TRASH_CAT, noise=True)

Number of data batch: 540
Number of batch: 8
Number of data batch: 80
Number of batch: 1


In [3]:
model = FeatureCNN()
model = model.cuda()

In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [5]:
# model.load(AE_CKPT_PATH)

for e in range(EPOCHS):
    
    train_loss = 0.0
    train_clf_acc = 0.0
    
    for x_batch_, y_batch_ in train_dloader.next_batch():
        x_batch_ = x_batch_.reshape(-1, IN_CHANNEL, HEIGHT, WIDTH)
        y_batch = np.repeat(y_batch_, 8, axis=0)
        
        x_batch = torch.FloatTensor(x_batch_).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        latent, logps = model(x_batch)
        loss = criterion(logps, y_batch)
        train_loss += loss.item()
        
        with torch.no_grad():
            ps = torch.exp(logps)
            val_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_clf_acc += torch.mean(equal.type(torch.FloatTensor))
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        
        valid_loss = 0.0
        valid_clf_acc = 0.0
        
        with torch.no_grad():
            model.eval()
            
            for x_batch_, y_batch_ in valid_dloader.next_batch():
                x_batch_ = x_batch_.reshape(-1, IN_CHANNEL, HEIGHT, WIDTH)
                y_batch = np.repeat(y_batch_, 8, axis=0)
                
                x_batch = torch.FloatTensor(x_batch_).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                latent, logps = model(x_batch)
                ps = torch.exp(logps)
                _, topk = ps.topk(1, dim=1)
                equal = topk == y_batch.view(*topk.size())
                valid_clf_acc += torch.mean(equal.type(torch.FloatTensor))
        
                loss = criterion(logps, y_batch)
                valid_loss += loss.item()

            train_loss /= len(train_dloader)
            train_clf_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_clf_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_clf_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_clf_acc:.8f}")

            model.train()
        
model.save(CNN_CKPT_PATH)

Epochs: 10/1000
Train loss: 1.32611130
Train acc: 0.37548828
Valid loss: 1.36254883
Valid acc: 0.42187500
Epochs: 20/1000
Train loss: 1.24887775
Train acc: 0.43994141
Valid loss: 1.38144720
Valid acc: 0.41406250
Epochs: 30/1000
Train loss: 1.27402252
Train acc: 0.42285156
Valid loss: 1.41140819
Valid acc: 0.41406250
Epochs: 40/1000
Train loss: 1.23372786
Train acc: 0.43652344
Valid loss: 1.35556328
Valid acc: 0.45703125
Epochs: 50/1000
Train loss: 1.23547833
Train acc: 0.43505859
Valid loss: 1.47463381
Valid acc: 0.42187500
Epochs: 60/1000
Train loss: 1.24880379
Train acc: 0.41699219
Valid loss: 1.56206298
Valid acc: 0.31640625
Epochs: 70/1000
Train loss: 1.21852699
Train acc: 0.45410156
Valid loss: 1.42551851
Valid acc: 0.42968750
Epochs: 80/1000
Train loss: 1.26438767
Train acc: 0.41552734
Valid loss: 1.42751265
Valid acc: 0.42187500
Epochs: 90/1000
Train loss: 1.22434156
Train acc: 0.44824219
Valid loss: 1.46611047
Valid acc: 0.41406250
Epochs: 100/1000
Train loss: 1.23366871
Train 

KeyboardInterrupt: 